In [3]:
from bs4 import BeautifulSoup
import requests


In [4]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(data,'lxml')

In [95]:
table = soup.find('table')

In [97]:
headers=[]
for th in table.find_all('th'):
    headers.append(th.text.split('\n')[0])
print ("table headers are {}".format(headers))
tablewidth = len(headers)
tablecontents = {}
for header in headers:
    tablecontents[header] = []
count = 1
for element in table.find_all('td'):
    tablecontents[headers[count%tablewidth-1]].append(element.text.split('\n')[0])
    count = count +1


table headers are ['Postcode', 'Borough', 'Neighbourhood']


In [98]:
import pandas as pd
can_df = pd.DataFrame(tablecontents)

In [99]:
can_df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [100]:
can_df_filter_NG = can_df[can_df['Borough']!='Not assigned'].reset_index(drop=True)

In [101]:
can_df_filter_NG.loc[can_df_filter_NG['Neighbourhood']=='Not assigned','Neighbourhood'] = can_df_filter_NG[can_df_filter_NG['Neighbourhood']=='Not assigned']['Borough']
can_df_filter_NG.groupby('Postcode')['Neighbourhood'].agg(lambda x: x.str.cat(sep=','))
can_df_agg = can_df_filter_NG.groupby('Postcode').last()
can_df_agg['Neighbourhood'] = can_df_filter_NG.groupby('Postcode')['Neighbourhood'].agg(lambda x: x.str.cat(sep=','))
can_df_agg.reset_index(inplace=True)
can_df_agg.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [102]:
can_df_agg.shape

(103, 3)